In [8]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from pipetorch.data import DFrame
from sklearn.linear_model import LinearRegression
import random

In [9]:
!ls /data/foodboost

ingredients.csv  nutritions.csv  recipes.csv  tags.csv


In [10]:
ingredients = pd.read_csv('/data/foodboost/ingredients.csv', index_col=[0]) # Importeren van alle datasets
nutritions = pd.read_csv('/data/foodboost/nutritions.csv', index_col=[0])
recipes = pd.read_csv('/data/foodboost/recipes.csv',index_col=[0])
tags = pd.read_csv('/data/foodboost/tags.csv', index_col=[0])
ingredients.head()

,recipe,ingredient,quantity,unit
0,Kruidnoten met choco-discodip,melkchocolade,100.0,g
1,Kruidnoten met choco-discodip,kruidnoten,100.0,g
2,Kruidnoten met choco-discodip,discodip,2.0,el
3,Kruidnoten in marsepein,blanke marsepein,150.0,g
4,Kruidnoten in marsepein,ongezouten roomboter,15.0,g


In [11]:
recipes.head()

,title,persons,time,calories,stars,url,image
0,Kruidnoten met choco-discodip,4,25,260,0,https://www.ah.nl/allerhande/recept/R-R1195893...,https://static.ah.nl/static/recepten/img_RAM_P...
0,Kruidnoten in marsepein,4,25,265,0,https://www.ah.nl/allerhande/recept/R-R1195892...,https://static.ah.nl/static/recepten/img_RAM_P...
0,Kruidnoten met chocodips,4,25,335,0,https://www.ah.nl/allerhande/recept/R-R1195891...,https://static.ah.nl/static/recepten/img_RAM_P...
0,Pepernotentaart met marsepeinstrik,10,30,560,0,https://www.ah.nl/allerhande/recept/R-R1195887...,https://static.ah.nl/static/recepten/img_RAM_P...
0,Perencake,12,10,265,0,https://www.ah.nl/allerhande/recept/R-R1195790...,https://static.ah.nl/static/recepten/img_RAM_P...


In [12]:
nutritions.head()

,recipe,nutrition,value
0,Kruidnoten met choco-discodip,energie,260 kcal
1,Kruidnoten met choco-discodip,koolhydraten,34 g
2,Kruidnoten met choco-discodip,waarvan suikers,22 g
3,Kruidnoten met choco-discodip,natrium,200 mg
4,Kruidnoten met choco-discodip,eiwit,3 g


In [13]:
tags.head()

,recipe,tag
0,Kruidnoten met choco-discodip,hollands
1,Kruidnoten met choco-discodip,gebak
2,Kruidnoten met choco-discodip,gebak
3,Kruidnoten met choco-discodip,sinterklaas
4,Kruidnoten met choco-discodip,sinterklaasavond


In [14]:
tags_new = tags.groupby('recipe')["tag"].apply(list) # Een dataframe met de tags voor elk recept
df_tags_new = pd.DataFrame(tags_new)
df_tags_new

,tag
recipe,
Andijviestamppot met gebakken salami,"[snel, hollands, stamppot, hoofdgerecht, wat e..."
Bietensoep met selderijroom,"[vooraf te maken, soep, voorgerecht, kerst, ko..."
Libanese wraps met appel-koolsalade,"[snel, wrap, hoofdgerecht, wat eten we vandaag]"
Plaattaart met gegrilde groenten en witte kaas,"[zonder vlees/vis, budget, oven, hoofdgerecht,..."
Sticky cauliflower (gegrilde bloemkoolroosjes) van Jord Althuizen,"[veganistisch, borrelhapje, tussendoortje, bij..."
...,...
Zweedse gehaktballetjes met room,"[scandinavisch, kindergerecht, bijgerecht, bak..."
Zweedse vegaballetjes met aardappel &amp; broccoli,"[vegetarisch, hoofdgerecht, wat eten we vandaa..."
Zwitserse kaasfondue,"[zonder vlees/vis, hoofdgerecht, fondue, sinte..."


In [15]:
def select_recept_meertags(*args):
    """
    Gebruiker kan zoveel tags opgeven als hij/zij wilt. 
    De functie geeft de recepten met deze tags.
    """
    aantal_recepten = []
    opgegeven_args = []
        
    for n in args:
        opgegeven_args.append(n)

    for row,column in df_tags_new.iterrows():
        result =  all(elem in column[0] for elem in opgegeven_args)
        if result == True:
            aantal_recepten.append(row)
    
    if len(aantal_recepten) > 0:
        return aantal_recepten
    else:
        return []

In [16]:
jo = select_recept_meertags('jamie oliver') # Alle gerechten met de tag 'jamie oliver'
oe = select_recept_meertags('oost-europees') # Alle gerechten met de tag 'oost-europees'
print(jo)
print(oe)

['Amuse zalm op een vorkje', 'Chocolade-kardemomcake', 'Eenvoudige rodekool', 'Frittata met lentegroenten', 'Gemengde geroosterde groenten', 'Romige paddenstoelensoep', 'Semifreddo met snickers', 'Simpele kaasfondue', 'Spi\xadna\xadzie-ri\xadcot\xadta\xadro\xadto\xadlo', 'Wellington van rundergehakt']
['Babka met chocolade en hazelnoten', 'Borsjt met dille en sour cream', 'Goulash met paprikamix', 'Goulash met rijst en broccoli', "Jamie Oliver's kip Kiev", 'Monique van Loons zoete-aardappellatkes met auberginesalade', 'Rijkgevulde goulashsoep', 'Stroganoff-schnitzel met bonensalade', 'Vegetarische koolraapgoulash met rijst', 'Zuurkoolschotel met worst en champignons']


In [17]:
df = pd.DataFrame({'ID':[], 'Recepten':[], 'Value':[]}) # Starten met een empty dataframe
control = True # Een boolean variable om steeds af te wisselen tussen jamie oliver en oost-europese gerechten
for i in range(100): # Ik maak 100 gebruikers aan                                   
    if control == True: 
        oe_new = random.sample(oe, k = 7) # Hier pak je 7 verschillende oost-europese gerechten
        control = False
        for recept in oe_new: # De recepten worden toegevoegd aan het dataframe
            new = pd.DataFrame({'ID':[i], 'Recepten':[recept], 'Value':[1]})
            df = pd.concat([df,new], ignore_index = True)
    else:
        jo_new = random.sample(jo, k = 7) # Hier pak je 7 verschillende gerechten van Jamie Oliver
        control = True
        for recept in jo_new: # De recepten worden toegevoegd aan het dataframe
            new = pd.DataFrame({'ID':[i], 'Recepten':[recept], 'Value':[1]})
            df = pd.concat([df,new], ignore_index = True)
df      

,ID,Recepten,Value
0,0.0,Borsjt met dille en sour cream,1.0
1,0.0,Goulash met rijst en broccoli,1.0
2,0.0,Stroganoff-schnitzel met bonensalade,1.0
3,0.0,Goulash met paprikamix,1.0
4,0.0,Jamie Oliver's kip Kiev,1.0
...,...,...,...
695,99.0,Simpele kaasfondue,1.0
696,99.0,Gemengde geroosterde groenten,1.0
697,99.0,Amuse zalm op een vorkje,1.0
698,99.0,Spi­na­zie-ri­cot­ta­ro­to­lo,1.0


In [18]:
p = df.pivot(index = 'ID', columns = 'Recepten', values = 'Value').fillna(0)
# Hier wordt een pivot table gemaakt, zodat er per gesimuleerde gebruiker zijn/haar lievelings gerechten op 1 rij staan.

In [20]:
p.head()

Recepten,Amuse zalm op een vorkje,Babka met chocolade en hazelnoten,Borsjt met dille en sour cream,Chocolade-kardemomcake,Eenvoudige rodekool,Frittata met lentegroenten,Gemengde geroosterde groenten,Goulash met paprikamix,Goulash met rijst en broccoli,Jamie Oliver's kip Kiev,Monique van Loons zoete-aardappellatkes met auberginesalade,Rijkgevulde goulashsoep,Romige paddenstoelensoep,Semifreddo met snickers,Simpele kaasfondue,Spi­na­zie-ri­cot­ta­ro­to­lo,Stroganoff-schnitzel met bonensalade,Vegetarische koolraapgoulash met rijst,Wellington van rundergehakt,Zuurkoolschotel met worst en champignons
ID,,,,,,,,,,,,,,,,,,,,
0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
2.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
3.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0


In [ ]:
p.to_csv('Simulatie jamie_oliver&oost_europees.csv', index=False) # Van het dataframe wordt een csv bestand gemaakt